## Adding Libraries, Importing Dataset.

In [11]:
options(dplyr.summarise.inform = FALSE)
options(warn = -1)
library(tidyverse)
library(baseballr)
library(ggthemes) # Themes for ggplot2.
library(ggimage) # Supports images in ggplot2.
library(gt) # Create tables.
library(readr) # Read in datasets.
library(scatterplot3d)
library(reshape)
library(ExcelFunctionsR)

In [3]:
# Custom plot settings.
theme_reach <- function() {
  theme_fivethirtyeight() +
    theme(
      legend.position = "none",
      plot.title = element_text(size = 22, hjust = 0.5, face = "bold"),
      plot.subtitle = element_text(size = 18, hjust = 0.5),
      plot.caption = element_text(size = 16),
      axis.title.x = element_text(size=18, margin = margin(t = 20)),
      axis.title.y = element_text(size=18, margin = margin(r = 20)),
      axis.text = element_text(size = 14),
      strip.text = element_text(size = 16, face = "bold"),
      legend.text = element_text(size = 14)
    )
}

In [4]:
all_data_2021 <- read.csv(file = "C:/Users/chris/Documents/Datasets/BaseballR/2021/2021merged.csv")

## Filter Dataset and Divide into Pre/Post-8-21-2021 Subsets.

In [46]:
logan_all <- all_data_2021 %>%
  filter(pitcher == 669302)

logan_pre <- logan_all %>%
  filter(game_date.y <= "2021-08-21") %>%
  filter(!is.na(release_spin_rate))
logan_post <- logan_all %>%
  filter(game_date.y > "2021-08-21") %>%
  filter(!is.na(release_spin_rate))

## Functions to calculate 3-dimensional distance from the average for each release point.

In [47]:
distance_pre = function(x, output){
  P = x["pitch_name"]
  A = x["release_pos_x"]
  B = x["release_pos_y"]
  C = x["release_pos_z"]
  D = 0
  D = ifelse(P == "4-Seam Fastball", (dist(rbind(zero_loc1, c(A, B, C)))), D)
  D = ifelse(P == "Changeup", (dist(rbind(zero_loc2, c(A, B, C)))), D)
  D = ifelse(P == "Knuckle Curve", (dist(rbind(zero_loc3, c(A, B, C)))), D)
  D = ifelse(P == "Slider", (dist(rbind(zero_loc4, c(A, B, C)))), D)
  
  return(D)
}
distance_post = function(x, output){
  P = x["pitch_name"]
  A = x["release_pos_x"]
  B = x["release_pos_y"]
  C = x["release_pos_z"]
  D = 0
  D = ifelse(P == "4-Seam Fastball", (dist(rbind(zero_loc_post1, c(A, B, C)))), D)
  D = ifelse(P == "Changeup", (dist(rbind(zero_loc_post2, c(A, B, C)))), D)
  D = ifelse(P == "Knuckle Curve", (dist(rbind(zero_loc_post3, c(A, B, C)))), D)
  D = ifelse(P == "Slider", (dist(rbind(zero_loc_post4, c(A, B, C)))), D)
  
  return(D)
}

## Calculating average release point by pitch for each subset.

In [48]:
logan_pre_grp <- logan_pre %>%
  group_by(pitch_name) %>%
  summarize(avg_release_x = mean(release_pos_x), 
            avg_release_y = mean(release_pos_y), 
            avg_release_z = mean(release_pos_z))

logan_post_grp <- logan_post %>%
  group_by(pitch_name) %>%
  summarize(avg_release_x = mean(release_pos_x), 
            avg_release_y = mean(release_pos_y), 
            avg_release_z = mean(release_pos_z))

zero_loc1 <- c(logan_pre_grp[1,]$avg_release_x, logan_pre_grp[1,]$avg_release_y, logan_pre_grp[1,]$avg_release_z)
zero_loc2 <- c(logan_pre_grp[2,]$avg_release_x, logan_pre_grp[2,]$avg_release_y, logan_pre_grp[2,]$avg_release_z)
zero_loc3 <- c(logan_pre_grp[3,]$avg_release_x, logan_pre_grp[3,]$avg_release_y, logan_pre_grp[3,]$avg_release_z)
zero_loc4 <- c(logan_pre_grp[4,]$avg_release_x, logan_pre_grp[4,]$avg_release_y, logan_pre_grp[4,]$avg_release_z)

zero_loc_post1 <- c(logan_post_grp[1,]$avg_release_x, logan_post_grp[1,]$avg_release_y, logan_post_grp[1,]$avg_release_z)
zero_loc_post2 <- c(logan_post_grp[2,]$avg_release_x, logan_post_grp[2,]$avg_release_y, logan_post_grp[2,]$avg_release_z)
zero_loc_post3 <- c(logan_post_grp[3,]$avg_release_x, logan_post_grp[3,]$avg_release_y, logan_post_grp[3,]$avg_release_z)
zero_loc_post4 <- c(logan_post_grp[4,]$avg_release_x, logan_post_grp[4,]$avg_release_y, logan_post_grp[4,]$avg_release_z)

## Calculating 3-dimensional change in average release points by pitch.

In [ ]:
diff1 <- dist(rbind(zero_loc1, c(logan_post_grp[1,]$avg_release_x, 
                                 logan_post_grp[1,]$avg_release_y, 
                                 logan_post_grp[1,]$avg_release_z)))
diff2 <- dist(rbind(zero_loc2, c(logan_post_grp[2,]$avg_release_x, 
                                 logan_post_grp[2,]$avg_release_y, 
                                 logan_post_grp[2,]$avg_release_z)))
diff3 <- dist(rbind(zero_loc3, c(logan_post_grp[3,]$avg_release_x, 
                                 logan_post_grp[3,]$avg_release_y, 
                                 logan_post_grp[3,]$avg_release_z)))
diff4 <- dist(rbind(zero_loc4, c(logan_post_grp[4,]$avg_release_x, 
                                 logan_post_grp[4,]$avg_release_y, 
                                 logan_post_grp[4,]$avg_release_z)))

## Computing Release Point Differences, 3-dimensional Movement, and Release Point Variance for Each Pitch.

In [49]:
logan_pre$release_point_diff <- apply(logan_pre, 1, distance_pre)
logan_post$release_point_diff <- apply(logan_post, 1, distance_post)

logan_pre$movement_abs <- sqrt((logan_pre$pfx_x ^ 2) + (logan_pre$pfx_z ^ 2))
logan_post$movement_abs <- sqrt((logan_post$pfx_x ^ 2) + (logan_post$pfx_z ^ 2))

variance_pre <- logan_pre %>%
  group_by(pitch_name) %>%
  summarize(variance = mean(release_point_diff ^2))

variance_post <- logan_post %>%
  group_by(pitch_name) %>%
  summarize(variance = mean(release_point_diff ^2))

## Computing Columns for Each Subset.

In [50]:
logan_pre <- logan_pre %>%
  group_by(pitch_name) %>%
  summarize(count = n(), 
            avg_spin_rate = mean(release_spin_rate), 
            spin_rate_stdev = sd(release_spin_rate),
            avg_mph = mean(release_speed), 
            mph_stdev = sd(release_speed),
            avg_release_x = mean(release_pos_x), 
            avg_release_y = mean(release_pos_y), 
            avg_release_z = mean(release_pos_z),
            release_x_stdev = sd(release_pos_x), 
            release_y_stdev = sd(release_pos_y), 
            release_z_stdev = sd(release_pos_z),
            avg_movement_x = mean(pfx_x), 
            movement_x_stdev = sd(pfx_x),
            avg_movement_z = mean(pfx_z), 
            movement_z_stdev = sd(pfx_z),
            avg_plate_x = mean(plate_x), 
            avg_plate_z = mean(plate_z),
            plate_x_stdev = sd(plate_x),
            plate_z_stdev = sd(plate_z),
            avg_release_diff = mean(release_point_diff),
            release_diff_stdev = sd(release_point_diff),
            avg_movement_abs = mean(movement_abs),
            movement_abs_stdev = sd(movement_abs))
logan_pre$date_range <- c("Through 8-21", "Through 8-21", "Through 8-21", "Through 8-21")

logan_post <- logan_post %>%
  group_by(pitch_name) %>%
  summarize(count = n(), 
            avg_spin_rate = mean(release_spin_rate), 
            spin_rate_stdev = sd(release_spin_rate),
            avg_mph = mean(release_speed), 
            mph_stdev = sd(release_speed),
            avg_release_x = mean(release_pos_x), 
            avg_release_y = mean(release_pos_y), 
            avg_release_z = mean(release_pos_z),
            release_x_stdev = sd(release_pos_x), 
            release_y_stdev = sd(release_pos_y), 
            release_z_stdev = sd(release_pos_z),
            avg_movement_x = mean(pfx_x), 
            movement_x_stdev = sd(pfx_x),
            avg_movement_z = mean(pfx_z), 
            movement_z_stdev = sd(pfx_z),
            avg_plate_x = mean(plate_x), 
            avg_plate_z = mean(plate_z),
            plate_x_stdev = sd(plate_x),
            plate_z_stdev = sd(plate_z),
            avg_release_diff = mean(release_point_diff),
            release_diff_stdev = sd(release_point_diff),
            avg_movement_abs = mean(movement_abs),
            movement_abs_stdev = sd(movement_abs))
logan_post$date_range <- c("After 8-21", "After 8-21", "After 8-21", "After 8-21")

## Adding Release Point Variance Column to our Tables.

In [51]:
logan_pre <- merge(logan_pre, variance_pre, by = "pitch_name")
logan_post <- merge(logan_post, variance_post, by = "pitch_name")

## Creating Table to Show Differences Between Subsets.

In [53]:
logan_diff <- data.frame(pitch_name = logan_pre$pitch_name)

diff_matrix <- data.matrix(subset(logan_post, select = -c(pitch_name, count, date_range))) - 
               data.matrix(subset(logan_pre, select = -c(pitch_name, count, date_range)))

logan_diff <- cbind(logan_diff, diff_matrix)
logan_diff$diff_3d <- c(diff1, diff2, diff3, diff4)

## Combine Data and Format Before Outputting to Table Visualization.

In [57]:
logan_all_by_pitch <- rbind(logan_pre, logan_post)

logan_statcast_table <- logan_all_by_pitch[, c(25, 1, 2, 5, 3, 7, 8, 9, 13, 15, 23)]
logan_statcast_table$avg_mph <- round(logan_statcast_table$avg_mph, 1)
logan_statcast_table$avg_spin_rate <- round(logan_statcast_table$avg_spin_rate)
logan_statcast_table$avg_release_x <- round(logan_statcast_table$avg_release_x, 2)
logan_statcast_table$avg_release_y <- round(logan_statcast_table$avg_release_y, 2)
logan_statcast_table$avg_release_z <- round(logan_statcast_table$avg_release_z, 2)
logan_statcast_table$avg_movement_x <- round(logan_statcast_table$avg_movement_x * 12, 2)
logan_statcast_table$avg_movement_z <- round(logan_statcast_table$avg_movement_z * 12, 2)
logan_statcast_table$avg_movement_abs <- round(logan_statcast_table$avg_movement_abs * 12, 2)
logan_statcast_table$date_range <- c("Through 8-21","","","","After 8-21","","","")

## Create and Save 'Statcast Data by Pitch' Table.

In [58]:
logan_statcast_table_gt <- logan_statcast_table %>%
  gt() %>%
  cols_label(date_range = "Date Range",
             pitch_name = "Pitch Name",
             count = "Pitch Count",
             avg_mph = "MPH",
             avg_spin_rate = "Spin Rate",
             avg_release_x = "X",
             avg_release_y = "Y",
             avg_release_z = "Z",
             avg_movement_x = "Horizontal",
             avg_movement_z = "Vertical",
             avg_movement_abs = "Absolute",) %>%
  cols_align(align = "center",
             columns = c(count, avg_mph, avg_spin_rate, avg_release_x, avg_release_y, avg_release_z, 
                         avg_movement_x, avg_movement_z, avg_movement_abs)) %>%
  gtExtras::gt_theme_espn() %>%
  tab_header(title = "Logan Gilbert Statcast Data by Pitch, 2021") %>%
  tab_style(style = cell_borders(sides = "right",
                                 color = "#D3D3D3",
                                 weight = px(2)),
            locations = cells_body(columns = c(pitch_name, avg_spin_rate, avg_release_z),
                                   rows = everything())) %>%
  tab_style(style = cell_borders(sides = "top",
                                 color = "#808080",
                                 weight = px(4)),
            locations = cells_body(columns = everything(),
                                   rows = 1)) %>%
  tab_style(style = cell_borders(sides = "bottom",
                                 color = "#808080",
                                 weight = px(4)),
            locations = cells_body(columns = everything(),
                                   rows = 4)) %>%
  tab_spanner(label = "Release Point (ft)", columns = c(avg_release_x, avg_release_y, avg_release_z)) %>%
  tab_spanner(label = "Spin Induced Movement (in)", columns = c(avg_movement_x, avg_movement_z, avg_movement_abs))

In [59]:
gtsave(logan_statcast_table_gt, "logan_statcast_table_gt.png")

## Format and Arrange 'Differences' Table. 

In [61]:
logan_diff$avg_movement_x <- ((logan_post$avg_movement_x-logan_pre$avg_movement_x)/logan_pre$avg_movement_x)*100
logan_diff$avg_movement_z <- ((logan_post$avg_movement_z-logan_pre$avg_movement_z)/logan_pre$avg_movement_z)*100
logan_diff$avg_movement_abs <- ((logan_post$avg_movement_abs-logan_pre$avg_movement_abs)/logan_pre$avg_movement_abs)*100
logan_diff$movement_abs_stdev <- ((logan_post$movement_abs_stdev-logan_pre$movement_abs_stdev)/logan_pre$movement_abs_stdev)*100
logan_diff$spin_rate_stdev <- ((logan_post$spin_rate_stdev-logan_pre$spin_rate_stdev)/logan_pre$spin_rate_stdev)*100
logan_diff$mph_stdev <- ((logan_post$mph_stdev-logan_pre$mph_stdev)/logan_pre$mph_stdev)*100
logan_diff$release_x_stdev <- ((logan_post$release_x_stdev-logan_pre$release_x_stdev)/logan_pre$release_x_stdev)*100
logan_diff$release_y_stdev <- ((logan_post$release_y_stdev-logan_pre$release_y_stdev)/logan_pre$release_y_stdev)*100
logan_diff$release_z_stdev <- ((logan_post$release_z_stdev-logan_pre$release_z_stdev)/logan_pre$release_z_stdev)*100
logan_diff$release_diff_stdev <- ((logan_post$release_diff_stdev-logan_pre$release_diff_stdev)/logan_pre$release_diff_stdev)*100
logan_diff$movement_x_stdev <- ((logan_post$movement_x_stdev-logan_pre$movement_x_stdev)/logan_pre$movement_x_stdev)*100
logan_diff$movement_z_stdev <- ((logan_post$movement_z_stdev-logan_pre$movement_z_stdev)/logan_pre$movement_z_stdev)*100
logan_diff$variance <- ((logan_post$variance-logan_pre$variance)/logan_pre$variance)*100

logan_diff$avg_mph <- as.character(round(logan_diff$avg_mph, 1))
logan_diff$avg_spin_rate <- as.character(round(logan_diff$avg_spin_rate, 0))
logan_diff$avg_release_x <- paste(as.character(round(logan_diff$avg_release_x * 12, 1)), "in", sep = " ")
logan_diff$avg_release_y <- paste(as.character(round(logan_diff$avg_release_y * 12, 1)), "in", sep = " ")
logan_diff$avg_release_z <- paste(as.character(round(logan_diff$avg_release_z * 12, 1)), "in", sep = " ")
logan_diff$avg_movement_x <- as.character(round(logan_diff$avg_movement_x, 0))
logan_diff$avg_movement_z <- as.character(round(logan_diff$avg_movement_z, 0))
logan_diff$avg_movement_abs <- as.character(round(logan_diff$avg_movement_abs, 0))
logan_diff$movement_abs_stdev <- as.character(round(logan_diff$movement_abs_stdev, 0))
logan_diff$spin_rate_stdev <- as.character(round(logan_diff$spin_rate_stdev, 0))
logan_diff$mph_stdev <- as.character(round(logan_diff$mph_stdev, 0))
logan_diff$release_x_stdev <- as.character(round(logan_diff$release_x_stdev, 0))
logan_diff$release_y_stdev <- as.character(round(logan_diff$release_y_stdev, 0))
logan_diff$release_z_stdev <- as.character(round(logan_diff$release_z_stdev, 0))
logan_diff$avg_release_diff <- paste(as.character(round(logan_diff$avg_release_diff * 12, 1)), "in", sep = " ")
logan_diff$release_diff_stdev <- as.character(round(logan_diff$release_diff_stdev, 0))
logan_diff$movement_x_stdev <- as.character(round(logan_diff$movement_x_stdev, 0))
logan_diff$movement_z_stdev <- as.character(round(logan_diff$movement_z_stdev, 0))
logan_diff$variance <- as.character(round(logan_diff$variance, 0))
logan_diff$diff_3d <- paste(as.character(round(logan_diff$diff_3d * 12, 1)), "in", sep = " ")

logan_diff_table <- logan_diff[, c(1, 4, 5, 2, 3, 6, 7, 8, 9, 10, 11, 12, 14, 13, 15, 22, 23, 25, 24)]

## Melt Table Twice to Arrange Vertically in Two Columns.

In [64]:
logan_melt <- melt(logan_diff_table, id = c("pitch_name"))

logan_melt1 <- logan_melt %>%
  filter(variable %in% c("mph_stdev", "spin_rate_stdev", "release_x_stdev", "release_y_stdev", "release_z_stdev",
                         "movement_x_stdev", "movement_z_stdev", "movement_abs_stdev", "variance"))
logan_melt1$value <- paste(as.character(logan_melt1$value), "%", sep = "")
logan_melt1$value <- ifelse(!grepl("-", logan_melt1$value), 
                            paste("+", as.character(logan_melt1$value), sep = ""),
                            logan_melt1$value)

logan_melt2 <- logan_melt %>%
  filter(variable %in% c("avg_mph", "avg_spin_rate", "avg_release_x", "avg_release_y", "avg_release_z",
                          "avg_movement_x", "avg_movement_z", "avg_movement_abs", "diff_3d"))
logan_melt2$value <- ifelse(logan_melt2$variable %in% c("avg_movement_x", "avg_movement_z", "avg_movement_abs"), 
                            paste(as.character(logan_melt2$value), "%", sep = ""),
                            logan_melt2$value)
logan_melt2$value <- ifelse((!grepl("-", logan_melt2$value)) & (logan_melt2$variable != "diff_3d"), 
                            paste("+", as.character(logan_melt2$value), sep = ""),
                            logan_melt2$value)
logan_melt2$value <- ifelse(logan_melt2$variable == "avg_mph", 
                            paste(as.character(logan_melt2$value), "mph", sep = " "),
                            logan_melt2$value)
logan_melt2$value <- ifelse(logan_melt2$variable == "avg_spin_rate", 
                            paste(as.character(logan_melt2$value), "rpm", sep = " "),
                            logan_melt2$value)

logan_diff_table2 <- cbind(logan_melt2, logan_melt1)
logan_diff_table2 <- logan_diff_table2[-4]

## Format Column Names. Arrange Rows before Outputting to Table.

In [66]:
logan_diff_table2$variable <- case_when(logan_diff_table2$variable == "avg_mph" ~ "MPH",
                                        logan_diff_table2$variable == "avg_spin_rate" ~ "Spin Rate",
                                        logan_diff_table2$variable == "avg_release_x" ~ "Release X",
                                        logan_diff_table2$variable == "avg_release_y" ~ "Release Y",
                                        logan_diff_table2$variable == "avg_release_z" ~ "Release Z",
                                        logan_diff_table2$variable == "diff_3d" ~ "\U0394 Release Point Absolute",
                                        logan_diff_table2$variable == "avg_movement_x" ~ "Movement X",
                                        logan_diff_table2$variable == "avg_movement_z" ~ "Movement Z",
                                        logan_diff_table2$variable == "avg_movement_abs" ~ "Movement")

logan_diff_table2$variable.1 <- case_when(logan_diff_table2$variable.1 == "mph_stdev" ~ "MPH \U03C3",
                                          logan_diff_table2$variable.1 == "spin_rate_stdev" ~ "Spin Rate \U03C3",
                                          logan_diff_table2$variable.1 == "release_x_stdev" ~ "Release Point X \U03C3",
                                          logan_diff_table2$variable.1 == "release_y_stdev" ~ "Release Point Y \U03C3",
                                          logan_diff_table2$variable.1 == "release_z_stdev" ~ "Release Point Z \U03C3",
                                          logan_diff_table2$variable.1 == "movement_x_stdev" ~ "Movement X \U03C3",
                                          logan_diff_table2$variable.1 == "movement_z_stdev" ~ "Movement Z \U03C3",
                                          logan_diff_table2$variable.1 == "movement_abs_stdev" ~ "Total Movement \U03C3",
                                          logan_diff_table2$variable.1 == "variance" ~ "Release Point Variance")

logan_diff_table2 <- rbind(logan_diff_table2[1:8,], 
                           logan_diff_table2[29:32,], 
                           logan_diff_table2[21:28,],
                           logan_diff_table2[9:20,],
                           logan_diff_table2[33:36,])

## Create and Save 'Differences' Table.

In [68]:
logan_diff_table2_gt <- logan_diff_table2 %>%
  gt(groupname_col = "pitch_name") %>%
  cols_label(variable.1 = "Standard Deviation",
             value.1 = "Value") %>%
  cols_align(align = "left",
             columns = c("variable", "value", "variable.1", "value.1")) %>%
  cols_align(align = "center",
             columns = c("value", "value.1")) %>%
  gtExtras::gt_theme_espn() %>%
  tab_header(title = "Logan Gilbert Pitch Comparison, 2021",
             subtitle = "Post-8-21-21 Data Shown in Relation to Pre-8-21-21 Data") %>%
  tab_options(table.width = pct(100)) %>%
  tab_style(style = cell_fill(color = "#90EE90"),
            locations = cells_body(columns = value,
                                   rows = (!grepl("-", value)) & !(variable %in% c("Release X", 
                                                                                   "Release Y", 
                                                                                   "Release Z",
                                                                                   "Movement X",
                                                                                   "Movement Z",
                                                                                   "\U0394 Release Point Absolute")))) %>%
  tab_style(style = cell_fill(color = "#ff726f"),
            locations = cells_body(columns = value,
                                   rows = (grepl("-", value)) & !(variable %in% c("Release X", 
                                                                                  "Release Y", 
                                                                                  "Release Z",
                                                                                  "Movement X",
                                                                                  "Movement Z",
                                                                                  "\U0394 Release Point Absolute")))) %>%
  tab_style(style = cell_fill(color = "#90EE90"),
            locations = cells_body(columns = value.1,
                                   rows = (grepl("-", value.1)) & !(variable.1 %in% c("Movement X \U03C3",
                                                                                      "Movement Z \U03C3",
                                                                                      "Release Point X \U03C3", 
                                                                                      "Release Point Y \U03C3", 
                                                                                      "Release Point Z \U03C3")))) %>%
  tab_style(style = cell_fill(color = "#ff726f"),
            locations = cells_body(columns = value.1,
                                   rows = (!grepl("-", value.1)) & !(variable.1 %in% c("Movement X \U03C3",
                                                                                       "Movement Z \U03C3",
                                                                                       "Release Point X \U03C3", 
                                                                                       "Release Point Y \U03C3", 
                                                                                       "Release Point Z \U03C3")) & (value.1 != "")))

In [69]:
gtsave(logan_diff_table2_gt, "logan_diff_table2_gt.png")

## Filter 'Differences' Table by Pitch to Create Four Smaller Tables.

In [70]:
logan_diff_fb <- logan_diff_table2 %>%
  filter(pitch_name == "4-Seam Fastball")
logan_diff_fb <- subset(logan_diff_fb, select = -c(pitch_name))

logan_diff_change <- logan_diff_table2 %>%
  filter(pitch_name == "Changeup")
logan_diff_change <- subset(logan_diff_change, select = -c(pitch_name))

logan_diff_curve <- logan_diff_table2 %>%
  filter(pitch_name == "Knuckle Curve")
logan_diff_curve <- subset(logan_diff_curve, select = -c(pitch_name))

logan_diff_slider <- logan_diff_table2 %>%
  filter(pitch_name == "Slider")
logan_diff_slider <- subset(logan_diff_slider, select = -c(pitch_name))

## Create 'Differences-FB' Table.

In [75]:
logan_diff_fb_gt <- logan_diff_fb %>%
  gt() %>%
  cols_label(variable.1 = "Standard Deviation",
             value.1 = "Value") %>%
  cols_align(align = "left",
             columns = c("variable", "value", "variable.1", "value.1")) %>%
  cols_align(align = "center",
             columns = c("value", "value.1")) %>%
  gtExtras::gt_theme_espn() %>%
  tab_header(title = "Logan Gilbert Fastball Comparison, 2021",
             subtitle = "Post-8-21-21 Data Shown in Relation to Pre-8-21-21 Data") %>%
  tab_options(table.width = pct(100)) %>%
  tab_style(style = cell_fill(color = "#90EE90"),
            locations = cells_body(columns = value,
                                   rows = (!grepl("-", value)) & !(variable %in% c("Release X", 
                                                                                   "Release Y", 
                                                                                   "Release Z",
                                                                                   "Movement X",
                                                                                   "Movement Z",
                                                                                   "\U0394 Release Point Absolute")))) %>%
  tab_style(style = cell_fill(color = "#ff726f"),
            locations = cells_body(columns = value,
                                   rows = (grepl("-", value)) & !(variable %in% c("Release X", 
                                                                                  "Release Y", 
                                                                                  "Release Z",
                                                                                  "Movement X",
                                                                                  "Movement Z",
                                                                                  "\U0394 Release Point Absolute")))) %>%
  tab_style(style = cell_fill(color = "#90EE90"),
            locations = cells_body(columns = value.1,
                                   rows = (grepl("-", value.1)) & !(variable.1 %in% c("Movement X \U03C3",
                                                                                      "Movement Z \U03C3",
                                                                                      "Release Point X \U03C3", 
                                                                                      "Release Point Y \U03C3", 
                                                                                      "Release Point Z \U03C3")))) %>%
  tab_style(style = cell_fill(color = "#ff726f"),
            locations = cells_body(columns = value.1,
                                   rows = (!grepl("-", value.1)) & !(variable.1 %in% c("Movement X \U03C3",
                                                                                       "Movement Z \U03C3",
                                                                                       "Release Point X \U03C3", 
                                                                                       "Release Point Y \U03C3", 
                                                                                       "Release Point Z \U03C3")) & (value.1 != "")))

## Create 'Differences-Change' Table.

In [73]:
logan_diff_change_gt <- logan_diff_change %>%
  gt() %>%
  cols_label(variable.1 = "Standard Deviation",
             value.1 = "Value") %>%
  cols_align(align = "left",
             columns = c("variable", "value", "variable.1", "value.1")) %>%
  cols_align(align = "center",
             columns = c("value", "value.1")) %>%
  gtExtras::gt_theme_espn() %>%
  tab_header(title = "Logan Gilbert Changeup Comparison, 2021",
             subtitle = "Post-8-21-21 Data Shown in Relation to Pre-8-21-21 Data") %>%
  tab_options(table.width = pct(100)) %>%
  tab_style(style = cell_fill(color = "#90EE90"),
            locations = cells_body(columns = value,
                                   rows = (!grepl("-", value)) & !(variable %in% c("Release X", 
                                                                                   "Release Y", 
                                                                                   "Release Z",
                                                                                   "Movement X",
                                                                                   "Movement Z",
                                                                                   "\U0394 Release Point Absolute")))) %>%
  tab_style(style = cell_fill(color = "#ff726f"),
            locations = cells_body(columns = value,
                                   rows = (grepl("-", value)) & !(variable %in% c("Release X", 
                                                                                  "Release Y", 
                                                                                  "Release Z",
                                                                                  "Movement X",
                                                                                  "Movement Z",
                                                                                  "\U0394 Release Point Absolute")))) %>%
  tab_style(style = cell_fill(color = "#90EE90"),
            locations = cells_body(columns = value.1,
                                   rows = (grepl("-", value.1)) & !(variable.1 %in% c("Movement X \U03C3",
                                                                                      "Movement Z \U03C3",
                                                                                      "Release Point X \U03C3", 
                                                                                      "Release Point Y \U03C3", 
                                                                                      "Release Point Z \U03C3")))) %>%
  tab_style(style = cell_fill(color = "#ff726f"),
            locations = cells_body(columns = value.1,
                                   rows = (!grepl("-", value.1)) & !(variable.1 %in% c("Movement X \U03C3",
                                                                                       "Movement Z \U03C3",
                                                                                       "Release Point X \U03C3", 
                                                                                       "Release Point Y \U03C3", 
                                                                                       "Release Point Z \U03C3")) & (value.1 != "")))

## Create 'Differences-Curve' Table.

In [76]:
logan_diff_curve_gt <- logan_diff_curve %>%
  gt() %>%
  cols_label(variable.1 = "Standard Deviation",
             value.1 = "Value") %>%
  cols_align(align = "left",
             columns = c("variable", "value", "variable.1", "value.1")) %>%
  cols_align(align = "center",
             columns = c("value", "value.1")) %>%
  gtExtras::gt_theme_espn() %>%
  tab_header(title = "Logan Gilbert Curveball Comparison, 2021",
             subtitle = "Post-8-21-21 Data Shown in Relation to Pre-8-21-21 Data") %>%
  tab_options(table.width = pct(100)) %>%
  tab_style(style = cell_fill(color = "#90EE90"),
            locations = cells_body(columns = value,
                                   rows = (!grepl("-", value)) & !(variable %in% c("Release X", 
                                                                                   "Release Y", 
                                                                                   "Release Z",
                                                                                   "Movement X",
                                                                                   "Movement Z",
                                                                                   "\U0394 Release Point Absolute")))) %>%
  tab_style(style = cell_fill(color = "#ff726f"),
            locations = cells_body(columns = value,
                                   rows = (grepl("-", value)) & !(variable %in% c("Release X", 
                                                                                  "Release Y", 
                                                                                  "Release Z",
                                                                                  "Movement X",
                                                                                  "Movement Z",
                                                                                  "\U0394 Release Point Absolute")))) %>%
  tab_style(style = cell_fill(color = "#90EE90"),
            locations = cells_body(columns = value.1,
                                   rows = (grepl("-", value.1)) & !(variable.1 %in% c("Movement X \U03C3",
                                                                                      "Movement Z \U03C3",
                                                                                      "Release Point X \U03C3", 
                                                                                      "Release Point Y \U03C3", 
                                                                                      "Release Point Z \U03C3")))) %>%
  tab_style(style = cell_fill(color = "#ff726f"),
            locations = cells_body(columns = value.1,
                                   rows = (!grepl("-", value.1)) & !(variable.1 %in% c("Movement X \U03C3",
                                                                                       "Movement Z \U03C3",
                                                                                       "Release Point X \U03C3", 
                                                                                       "Release Point Y \U03C3", 
                                                                                       "Release Point Z \U03C3")) & (value.1 != "")))

## Create 'Differences-Slider' Table.

In [77]:
logan_diff_slider_gt <- logan_diff_slider %>%
  gt() %>%
  cols_label(variable.1 = "Standard Deviation",
             value.1 = "Value") %>%
  cols_align(align = "left",
             columns = c("variable", "value", "variable.1", "value.1")) %>%
  cols_align(align = "center",
             columns = c("value", "value.1")) %>%
  gtExtras::gt_theme_espn() %>%
  tab_header(title = "Logan Gilbert Slider Comparison, 2021",
             subtitle = "Post-8-21-21 Data Shown in Relation to Pre-8-21-21 Data") %>%
  tab_options(table.width = pct(100)) %>%
  tab_style(style = cell_fill(color = "#90EE90"),
            locations = cells_body(columns = value,
                                   rows = (!grepl("-", value)) & !(variable %in% c("Release X", 
                                                                                   "Release Y", 
                                                                                   "Release Z",
                                                                                   "Movement X",
                                                                                   "Movement Z",
                                                                                   "\U0394 Release Point Absolute")))) %>%
  tab_style(style = cell_fill(color = "#ff726f"),
            locations = cells_body(columns = value,
                                   rows = (grepl("-", value)) & !(variable %in% c("Release X", 
                                                                                  "Release Y", 
                                                                                  "Release Z",
                                                                                  "Movement X",
                                                                                  "Movement Z",
                                                                                  "\U0394 Release Point Absolute")))) %>%
  tab_style(style = cell_fill(color = "#90EE90"),
            locations = cells_body(columns = value.1,
                                   rows = (grepl("-", value.1)) & !(variable.1 %in% c("Movement X \U03C3",
                                                                                      "Movement Z \U03C3",
                                                                                      "Release Point X \U03C3", 
                                                                                      "Release Point Y \U03C3", 
                                                                                      "Release Point Z \U03C3")))) %>%
  tab_style(style = cell_fill(color = "#ff726f"),
            locations = cells_body(columns = value.1,
                                   rows = (!grepl("-", value.1)) & !(variable.1 %in% c("Movement X \U03C3",
                                                                                       "Movement Z \U03C3",
                                                                                       "Release Point X \U03C3", 
                                                                                       "Release Point Y \U03C3", 
                                                                                       "Release Point Z \U03C3")) & (value.1 != "")))

## Save 'Differences by Pitch' Tables.

In [78]:
gtsave(logan_diff_fb_gt, "logan-diff-fb-gt.png")
gtsave(logan_diff_change_gt, "logan-diff-change-gt.png")
gtsave(logan_diff_curve_gt, "logan-diff-curve-gt.png")
gtsave(logan_diff_slider_gt, "logan-diff-slider-gt.png")